In [12]:
#nltk.download()
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [3]:
df = pd.read_csv("Reviews.csv")
df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [4]:
df = df.drop(columns=['Id', 'Time'])
df.head()

,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Summary,Text
0,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,"""Delight"" says it all",This is a confection that has been around a fe...
3,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,Cough Medicine,If you are looking for the secret ingredient i...
4,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,Great taffy,Great taffy at a great price. There was a wid...


In [6]:
cols = ['ProductId', 'UserId', 'ProfileName', 'Summary']
for col in cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])

df.head()

,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Summary,Text
0,27619,188646,190356,1,1,5,83434,I have bought several of the Vitality canned d...
1,72383,25105,190863,0,0,1,167649,Product arrived labeled as Jumbo Salted Peanut...
2,15267,210482,128381,1,1,4,126,This is a confection that has been around a fe...
3,19718,152635,92148,3,3,2,47071,If you are looking for the secret ingredient i...
4,69007,57804,120309,0,0,5,107323,Great taffy at a great price. There was a wid...


In [10]:
df['Text'] = df['Text'].str.lower()
df['Text']

0         i have bought several of the vitality canned d...
1         product arrived labeled as jumbo salted peanut...
2         this is a confection that has been around a fe...
3         if you are looking for the secret ingredient i...
4         great taffy at a great price.  there was a wid...
                                ...                        
568449    great for sesame chicken..this is a good if no...
568450    i'm disappointed with the flavor. the chocolat...
568451    these stars are small, so you can give 10-15 o...
568452    these are the best treats for training and rew...
568453    i am very satisfied ,product is as advertised,...
Name: Text, Length: 568454, dtype: object

In [11]:
df['Text'] = df['Text'].apply(word_tokenize)
df['Text']

0         [i, have, bought, several, of, the, vitality, ...
1         [product, arrived, labeled, as, jumbo, salted,...
2         [this, is, a, confection, that, has, been, aro...
3         [if, you, are, looking, for, the, secret, ingr...
4         [great, taffy, at, a, great, price, ., there, ...
                                ...                        
568449    [great, for, sesame, chicken, .., this, is, a,...
568450    [i, 'm, disappointed, with, the, flavor, ., th...
568451    [these, stars, are, small, ,, so, you, can, gi...
568452    [these, are, the, best, treats, for, training,...
568453    [i, am, very, satisfied, ,, product, is, as, a...
Name: Text, Length: 568454, dtype: object

In [13]:
stop_words = set(stopwords.words("english"))
print(stop_words)

{"you'll", 'between', 'most', 'from', 'no', 'these', 't', 'after', 'will', 'now', "it's", 'ourselves', 'aren', 'himself', 'in', 'he', 'more', 'nor', 'o', 'off', 'doesn', 'below', 'there', 'wasn', 'itself', 'mustn', 'again', 'can', 'theirs', 'of', 'didn', 'hasn', 'until', "haven't", 'shan', 'any', "you'd", 'having', 'only', "didn't", 'hers', 'while', 'too', 'with', 'my', 'doing', 'they', 'won', 'weren', 's', 'so', 'under', 'and', 'd', 'it', 'that', 'which', 'haven', 'yourselves', "mustn't", 'had', 'does', "hasn't", 'should', 'into', 'where', 'very', 'not', 'hadn', "that'll", 'am', 'few', 'is', 'myself', 'his', 'been', 'don', 'do', 'themselves', 'how', 'then', 'him', 'what', 'other', 'same', "mightn't", 'all', 're', 'some', 'by', 'own', 'your', 'her', 'above', 'ma', 'if', "shouldn't", 'further', 'on', "weren't", 'who', 'them', "shan't", 'during', "won't", 'this', 'against', 'shouldn', 'm', 'you', 'each', 'me', 'are', 'being', 'their', 'at', 'both', 'a', 'about', 'just', 'wouldn', "she's"